### VIDEO PACKAGING

Now that we have the videos in the desired qualities the next step is make the dash manifest, knows as Media Presentation Description schema. The estructure of this file is defined in an ISO standard that can be found here [MPD schema](http://standards.iso.org/ittf/PubliclyAvailableStandards/MPEG-DASH_schema_files/DASH-MPD.xsd).

Luckily there are open source tools for doing this, you can use ffmpeg as we did in the [video encoding notebook](1 - Video Encoding.ipynb). The first aproach that I wook was with [MP4 Box](https://gpac.wp.imt.fr/mp4box/) but I needed to modify manually the XML files afterwards so I change the tool to [Shaka-Packager](https://github.com/google/shaka-packager)

For that reason in this notebook we are going to package the videos created in the VIDEO ENCODING following the [recomendations](https://google.github.io/shaka-packager/html/tutorials/dash.html) from Google and using the Shaka Packager

Package the qualities in the DASH specifications 

The command that for the mp4 encoding that we already did is the following.

` packager \
  in=h264_baseline_360p_600.mp4,stream=audio,output=audio.mp4 \
  in=h264_baseline_360p_600.mp4,stream=video,output=h264_360p.mp4 \
  in=h264_main_480p_1000.mp4,stream=video,output=h264_480p.mp4 \
  in=h264_main_720p_3000.mp4,stream=video,output=h264_720p.mp4 \
  in=h264_high_1080p_6000.mp4,stream=video,output=h264_1080p.mp4 \
  --mpd_output h264.mpd`
  
 The first thing that we see is that it use one off the video to link the audio file, when going trought the [MDN DASH ref](https://developer.mozilla.org/en-US/docs/Web/HTML/DASH_Adaptive_Streaming_for_HTML_5_Video)
 documentation they first create an extra file for the audio. Shaka packager avoid this extra step stracting the audio from one of the videos.
 
 You should modify the command for the packager to meet your requirements but the process is esay an really straighforward.
 
 

In [2]:
## We are going to use shaka packager windows binaries and run the commands refering to the path in our local directory

import os
import sys
import subprocess as sp

shaka_path = os.path.join(os.getcwd(),'shaka_packager','packager-win.exe')
video_folder = os.path.join(os.getcwd(),'output_dir')
output_dir_path = os.path.join(os.getcwd(),'video_packaged')

print('shaka_path = ', shaka_path)
print('video_folder = ', video_folder)
print('output_dir_path = ', output_dir_path)


shaka_path =  D:\mp4dash\shaka_packager\packager-win.exe
video_folder =  D:\mp4dash\output_dir
output_dir_path =  D:\mp4dash\video_packaged


In [3]:
def packager_command(video_folder,output_dir_path, output_name,shaka_path):
    output_complete = os.path.join(output_dir_path, output_name)
    input_complete = os.path.join(video_folder, 'google_')
    
    string = shaka_path + ' in={video_name}_baseline_360p_600.mp4,stream=audio,output={output_complete}_audio.mp4' + \
                          ' in={video_name}_baseline_360p_600.mp4,stream=video,output={output_complete}_360p.mp4' + \
                          ' in={video_name}_baseline_480p_1000.mp4,stream=video,output={output_complete}_480p.mp4' + \
                          ' in={video_name}_baseline_720p_1500.mp4,stream=video,output={output_complete}_720p.mp4' + \
                          ' in={video_name}_baseline_1080p_3000.mp4,stream=video,output={output_complete}_1080p.mp4' + \
                          ' --mpd_output {output_complete}_manifest.mpd'

    formated_string =string.format(video_name=input_complete, output_complete=output_complete)
    return formated_string

In [4]:
command = packager_command(video_folder,output_dir_path,'google_',shaka_path)

In [68]:
## RUN THE PACKAGE COMMAND
import time
# get the time
start_time = time.time()
# run the command
command_output = sp.check_output(command).decode()
#calculate the elapsed_time
elapsed_time = time.time() - start_time
print('video package finished')
print('elapsed_time: ', elapsed_time)


video package finished
elapsed_time:  103.55950951576233


### VIDEO ANALYSIS

We can  use the shaka-packager instead of ffprobe as we did before for analize the video metadata [ref](https://google.github.io/shaka-packager/html/tutorials/basic_usage.html#media-file-analysis)

`packager input=some_content.mp4 --dump_stream_info `

Doing the analysis for the 540p video is as simple as change the input path to the path of our video.

In [9]:
# Define path to the video
video_path = os.path.join(output_dir_path,'output_name960x540.mp4')
# Definition of analysis the command
analysis_command = '{} input={} --dump_stream_info'.format(shaka_path,video_path)
print(analysis_command)
# Run the command
out =  sp.check_output(analysis_command).decode()
print(out)

D:\mp4dash\shaka_packager\packager-win.exe input=D:\mp4dash\video_packaged\output_name960x540.mp4 --dump_stream_info

File "D:\mp4dash\video_packaged\output_name960x540.mp4":
Found 2 stream(s).
Stream [0] type: Video
 codec_string: avc1.64001f
 time_scale: 12800
 duration: 67584 (5.3 seconds)
 is_encrypted: false
 codec: H264
 width: 960
 height: 540
 pixel_aspect_ratio: 1:1
 trick_play_rate: 0
 nalu_length_size: 4

Stream [1] type: Audio
 codec_string: mp4a.40.2
 time_scale: 48000
 duration: 256000 (5.3 seconds)
 is_encrypted: false
 codec: AAC
 sample_bits: 16
 num_channels: 2
 sampling_frequency: 48000
 language: und
 seek_preroll_ns: 20833

Packaging completed successfully.



### CONCLUSION

The time that shaka-packager spend in a video of 1 hour was ~100 seconds so it seems that fit perfectly with our requeriments

##### NEXT STEP

1. Set up a local server to play the streamed videos.